In [1]:
import pyximport; pyximport.install()
from gpcode import *

import random
import operator
import itertools
import marshal
import multiprocessing as mp
import time, datetime, os
from deap import creator, gp, base, tools, algorithms

DEFAULT_TREE_DEPTH = 5
machine_alias = "Enchilada"

In [4]:
def generateAllHeuristicTrees(depth=1, ands=0):
    main = "IfThenElse("
    #one_parameter_functions = ["isSameSuit", "hasDoubles", "notSameSuit", "notHasDoubles"]
    #two_parameter_functions = ["cardDifferenceLE", "cardDifferenceGE", "highestCardGE", "highestCardLE", "lowestCardGE", "lowestCardLE"]
    one_parameter_functions = []
    two_parameter_functions = ["highestCardGE", "highestCardLE", "lowestCardGE", "lowestCardLE"]
    two_parameter_functions_arg1 = ["totalPotGE", "totalPotLE"]
    three_parameter_functions = ["handValueGE", "handValueLE"]
    int_parameters = range(2, 15)
    int_parameters_arg1 = range(1, 33)
    int_parameters_arg2 = range(1, 11)
    string_parameters = ["'raise'", "'check'", "'fold'"]
    special_operations = ["and_"]

    first_param = []
    
    for func in one_parameter_functions:
        result = func + '(ARG0)'
        first_param.append(result)
    
    for func in two_parameter_functions:
        for param_int in int_parameters:
            result = func + '(ARG0,'
            result += str(param_int) + ')'
            first_param.append(result)

    for func in two_parameter_functions_arg1:
        for param_int in int_parameters_arg1:
            result = func + '(ARG1,'
            result += str(param_int) + ')'
            first_param.append(result)

    for func in three_parameter_functions:
        for param_int in int_parameters_arg2:
            result = func + '(ARG0,'
            result += str(param_int) + ',ARG2)'
            first_param.append(result)

    '''
    temp = []
    for func1 in first_param:
        for func2 in first_param:
            if func1.split("(")[0] != func2.split("(")[0]:
                temp.append("and_(" + func1 + "," + func2 + ")")
    
    first_param.extend(temp)
    '''
                
    second_param = list(string_parameters)
    
    third_param = []
    if depth == 1:
        third_param = list(string_parameters)
    else:
        third_param = generateAllHeuristicTrees(depth-1)
    
    trees = []
    if ands == 0:
        for x in first_param:
            for y in second_param:
                for z in third_param:
                    trees.append(main + x + ',' + y + ',' + z + ')')
    elif ands == 1:
        for i in range(0, len(first_param)-1):
            for j in range(i+1, len(first_param)):
                for y in second_param:
                    for z in third_param:
                        trees.append(main + 'and_(' + first_param[i] + ',' + first_param[j] + '),' + y + ',' + z + ')')
    
    return trees

def addStringToHeuristic(heuristic_str, new_statement_str):
    comma_pos = heuristic_str.rfind(',')
    comma_pos2 = new_statement_str.rfind(',')

    return heuristic_str[0:comma_pos+2] + new_statement_str[0:comma_pos2] + heuristic_str[comma_pos:] + ")"

def generateListPrimitiveTreesFromListStrings(list_strings, default=None):
    result = []
    for tree_string in list_strings:
        if default == None:
            result.append(gp.PrimitiveTree.from_string(tree_string, pset))
        else:
            result.append(gp.PrimitiveTree.from_string(addStringToHeuristic(default, tree_string), pset))
    
    return result

def findComplexity(ind):
    complexity = 0
    
    for expr in ind:
        if expr.name != "IfThenElse" and expr.name != "and_":
            if isinstance(expr, gp.Terminal):
                if expr.name == "raise" or expr.name == "fold" or expr.name == "check":
                    complexity += 1
            else:
                complexity += 1
    
    return complexity

In [ ]:
num_sims = 80000 * 5

filename = "Exhaustive Search/exs-output-" + machine_alias + "-" + str(datetime.datetime.now()) +  ".txt"
createLogFile(filename, "Exhaustive Search\nNumber of Simulations: " + str(num_sims) + "\n")

temp_pop = generateListPrimitiveTreesFromListStrings(generateAllHeuristicTrees(1, 1), None)
print len(temp_pop)
to_remove = []
for p in temp_pop:
    if not isTreeValid(p, all_hands):
        to_remove.append(p)
    else:
        temp_p = simplifyByRemovingStatements(simplifyTree(p, all_hands), all_hands)
        if findComplexity(p) != findComplexity(temp_p):
            to_remove.append(p)

#for p in to_remove:
#    pop.remove(p)
pop = []
for p in temp_pop:
    if not isThereCopyOfIndividual(pop, p) and p not in to_remove:
        pop.append(p)

print len(pop)

toolbox.register("evaluate", evaluate, n=num_sims)

pop = findFitness(pop)

for x in toolbox.select(pop, 20):
    logToFile(filename, str(x) + ";" + str(x.fitness) + ";\n")
    print x
    print "Fitness: " + str(x.fitness)

82620


In [ ]:
indv = []
indv.append(gp.PrimitiveTree.from_string("'raise'", pset))
indv.append(gp.PrimitiveTree.from_string("'check'", pset))
indv.append(gp.PrimitiveTree.from_string("IfThenElse(and_(lowestCardLE(ARG0, 7), highestCardLE(ARG0, 11)), 'check', IfThenElse(and_(totalPotGE(ARG1, 3), notHasDoubles(ARG0)), 'check', 'raise'))",pset))
indv.append(gp.PrimitiveTree.from_string("IfThenElse(and_(lowestCardLE(ARG0, 7), highestCardLE(ARG0, 11)), 'check', IfThenElse(totalPotLE(ARG1, 2), 'raise', IfThenElse(hasDoubles(ARG0), 'raise', 'check')))",pset))
indv.append(gp.PrimitiveTree.from_string("IfThenElse(and_(lowestCardLE(ARG0, 7), highestCardLE(ARG0, 11)), 'check', IfThenElse(totalPotLE(ARG1, 2), 'raise', 'check'))",pset))

toolbox.register("evaluate", evaluate, n=80000 * 5)

indv = findFitness(indv)

for i in indv:
    print i
    print i.fitness


In [8]:
indv = gp.PrimitiveTree.from_string("IfThenElse(and_(lowestCardLE(ARG0, 7), highestCardLE(ARG0, 11)), 'check', IfThenElse(and_(hasDoubles(ARG0), lowestCardGE(ARG0, 5)), 'raise', 'check'))", pset)

print simplifyTree(indv, all_hands)
print simplifyByRemovingStatements(indv, all_hands)

IfThenElse(and_(lowestCardLE(ARG0, 7), highestCardLE(ARG0, 11)), 'check', IfThenElse(and_(hasDoubles(ARG0), lowestCardGE(ARG0, 5)), 'raise', 'check'))
IfThenElse(and_(lowestCardLE(ARG0, 7), highestCardLE(ARG0, 11)), 'check', IfThenElse(and_(hasDoubles(ARG0), lowestCardGE(ARG0, 5)), 'raise', 'check'))
